In [18]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from openpyxl import Workbook
import time
import sys
import os


now = time.localtime()
s = '%04d년 %02d월 %02d일 %02d시 %02d분 %02d초' % (
    now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)


keyword = input("검색할 키워드를 입력하세요: ")
num_videos = int(input("검색할 영상의 개수를 입력하세요: "))
num_comments = int(input("추출할 댓글의 개수를 입력하세요: "))
file_path = input("크롤링한 결과를 저장할 경로를 입력하세요(예: C:/py_temp/): ")
save_txt = f"{file_path}{s} {keyword}.txt"
save_csv = f"{file_path}{s} {keyword}.csv"
save_xlsx = f"{file_path}{s} {keyword}.xlsx"


if not os.path.exists(file_path):
    print(f"입력하신 경로인 {file_path} 가 존재하지 않아 생성 후 크롤링 작업을 진행합니다.")
    os.makedirs(file_path)
else:
    print(f"입력한 경로인 {file_path} 가 존재하어 바로 크롤링 작업을 시작하겠습니다.")


print("웹드라이버 설정 시작")
path = ChromeDriverManager().install()
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
print("웹드라이버 설정 완료")


driver.get(f"https://www.youtube.com/results?search_query={keyword}")


video_links = []
while len(video_links) < num_videos:
    
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    driver.implicitly_wait(3)
    time.sleep(3) 

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    
    for link in soup.find_all('a', {'id': 'video-title'}):
        video_links.append("https://www.youtube.com" + link['href'])


data = []


orig_stdout = sys.stdout
f = open(save_txt, 'a', encoding="UTF-8")
sys.stdout = f


num = int(0)
for video_url in video_links[:num_videos]:
    driver.get(video_url)

    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    
    comments = soup.find_all('yt-formatted-string', {'id': 'content-text'})
    comment_authors = soup.find_all('a', {'id': 'author-text'})
    comment_timestamps = soup.find_all('yt-formatted-string', {'class': 'published-time-text'})
    num_comments_actual = min(num_comments, len(comments), len(comment_authors), len(comment_timestamps))

    for i in range(num_comments_actual):
        comment = comments[i]
        author = comment_authors[i].text.strip()
        timestamp = comment_timestamps[i].text
        content = comment.text

        print("-" * 30)
        print("영상 URL: ", video_url)
        print("댓글 작성자명: ", author)
        print("댓글 작성시간: ", timestamp)
        print("댓글 내용: ", content)
        print("-" * 30)
        print("\n")

        
        data.append([num, video_url, author, timestamp, content])

        num += 1


sys.stdout = orig_stdout
f.close()


driver.quit()


df = pd.DataFrame(data, columns=['번호', '영상 URL', '댓글 작성자명', '댓글 작성시간', '댓글 내용'])


df.to_csv(save_csv, index=False, header=True)


df.to_excel(save_xlsx, index=False, header=True)

print("데이터 저장 완료")

검색할 키워드를 입력하세요: 일본여행
검색할 영상의 개수를 입력하세요: 10
추출할 댓글의 개수를 입력하세요: 10
크롤링한 결과를 저장할 경로를 입력하세요(예: C:/py_temp/): D:\dava\
입력한 경로인 D:\dava\ 가 존재하어 바로 크롤링 작업을 시작하겠습니다.
웹드라이버 설정 시작


The version of chrome cannot be detected. Trying with latest driver version


웹드라이버 설정 완료
데이터 저장이 완료되었습니다.
